### Implementación del algoritmo genético para minimizar funciones continuas con dominio definido  

**Pablo Blanco | A01637761**  

Código basado en: www.machinelearningmastery.com

**Importar las librerías necesarias**

In [1]:
import numpy as np

**Function that helps us change from a list of bits (that seems like a bitstring) to a float**

In [2]:
def __decode(bounds: np.array, number_of_bits: int, bitstring: list) -> float:
    """Decode a bitstring into a float in the given bounds.

    Args:
        bounds (np.array): minimum and maximum values of the floats in the function
        number_of_bits (int): number of bits in the bitstring
        bitstring (str): the bitstring to decode as a list

    Returns:
        float: float from the bitstring
    """
    decoded = []
    largest_posible = 2 ** number_of_bits
    
    # for each vairable that has bounds
    for i in range(len(bounds)):
        
        # extract the substring
        start, end = i * number_of_bits, (number_of_bits * (i + 1))
        substring = bitstring[start:end]
        
        # convert bitstring to a string of chars
        chars = ''.join([str(s) for s in substring])
        
        # convert string to integer
        integer = int(chars, 2)
        
        # scale integer to desired range
        value = bounds[i][0] + (integer / largest_posible) * (bounds[i][1] - bounds[i][0])
        
        # store
        decoded.append(value)
        
    return decoded

**Selection of fittest parents**

In [3]:
def __selection(population: np.array, scores: list, k: int = 5) -> list:
    """Regresa al mejor elemento de un torneo de k contrincantes

    Args:
        population (list): La población completa
        scores (list): Los puntajes de cada individuo
        k (int, optional): El número de contrincantes a enfrentar. Defaults to 3.

    Returns:
        list: El mejor individuo de la población que participó dentro del torneo
    """
    # Take an element from the population at random
    element_number = np.random.randint(len(population))
    
    for enemy in np.random.randint(0, len(population), k):
        # check if the enemy's score is better (e.g. perform a tournament)
        if scores[enemy] < scores[element_number]:
            element_number = enemy

    # return the best element
    return population[element_number]

**Recombination function**

In [4]:
def __crossover(parent_1: list, parent_2: list, crossover_rate: float) -> np.array:
    """Cross over two parents to create two children

    Args:
            parent_1 (list): Parent 01
            parent_2 (list): Parent 02
            r_cross (float): Crossover rate

    Returns:
            np.array: Array of 2 children
    """
    # create empty array
    newborns = np.zeros((2, len(parent_1)), dtype=list)
    
    newborns[0] = parent_1.copy()
    newborns[1] = parent_2.copy()

    # check for recombination rate
    if np.random.rand() < crossover_rate:

        # select crossover point that is not on the end of the strin
        midpoint = np.random.randint(1, len(parent_1) - 2)

        # perform crossover
        newborns[0] = np.concatenate((parent_1[:midpoint], parent_2[midpoint:]))
        newborns[1] = np.concatenate((parent_2[:midpoint], parent_1[midpoint:]))

    return newborns


**Mutation function**

In [5]:
def __mutation(bitstring: list, mutation_rate: float):
    """Create a mutation in a given bitstring.

    Args:
        bitstring (list): A list of bits.
        mutation_rate (float): The rate at which there may be a mutation.
    """
    for i in range(len(bitstring)):
        
        # check for a mutation
        if np.random.rand() < mutation_rate:
            
            # flip the bit
            bitstring[i] = 1 - bitstring[i]


**Function that runs the algorithm for a function given**

In [6]:
def genetic_algorithm(func, bounds: np.array, number_of_bits: int, number_of_iterations: int, population_size: int, crossover_rate: float, mutation_rate: float, verbose: bool = False) -> float and float:
    """Performs the genetic algorithm to find the minimum of a function given.

    Args:
        func: The function to minimize.
        bounds (np.array): The bounds of the domain of the function.
        number_of_bits (int): Number of bits to consider in each chromosome.
        number_of_iterations (int): Number of iterations to perform.
        population_size (int): Number of individuals in the population.
        crossover_rate (float): The rate at which crossover occurs.
        mutation_rate (float): The rate at which mutation occurs.
        verbose (bool, optional): If the function should print how it is performing. Defaults to False.

    Returns:
        float: best number
        float: best cost
    """
    global __decode, __selection, __crossover, __mutation
    
    # chromosome length
    chromosome_length = number_of_bits * len(bounds)

    # initial population of random bitstring
    population = [np.random.randint(0, 2, chromosome_length).tolist() for _ in range(population_size)]

    # keep track of best solution
    best, best_eval = 0, func(__decode(bounds, number_of_bits, population[0]))

    # enumerate generations
    for generation in range(number_of_iterations):

        # decode all elements in population
        decoded_population = [__decode(bounds, number_of_bits, element) for element in population]

        # evaluate all candidates in the population
        scores = [func(decoded_element) for decoded_element in decoded_population]

        # check for new best solution
        for i in range(population_size):
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
                if verbose:
                    print(f"GEN: {generation}. New best f({decoded_population[i]}) = {scores[i]}")

        # select parents for recombination
        selected_parents = [__selection(population, scores) for _ in range(population_size)]

        # create the next generation
        children = np.empty((population_size, chromosome_length), dtype=list)
        children_number = 0

        for i in range(0, population_size, 2):

            # get selected parents in pairs
            parent_1, parent_2 = selected_parents[i], selected_parents[i+1]

            # crossover and mutation
            for child in __crossover(parent_1, parent_2, crossover_rate):
                # mutation
                __mutation(child, mutation_rate)

                # store for next generation
                children[children_number] = child

                children_number += 1

        # replace population
        population = children

    return best, best_eval


**Define minimizing function and run the algorithm**

In [7]:
def f(x: list) -> float:
    return (x[0] ** 4) / 4 + (x[0] ** 3) / 3 - (x[0] ** 2)

In [8]:
# define range for input
bounds = np.array([[-5.0, 5.0]])

# define the total iterations
number_of_iterations = 500

# bits per variable
number_of_bits = 16

# define the population size
population_size = 500

# crossover rate
crossover_rate = 0.9

# mutation rate
mutation_rate = 1.0 / (float(number_of_bits) * len(bounds))

# perform the genetic algorithm search
best, score = genetic_algorithm(func=f, bounds=bounds, number_of_bits=number_of_bits, number_of_iterations=number_of_iterations, population_size=population_size,
                                crossover_rate=crossover_rate, mutation_rate=mutation_rate, verbose=False)

decoded = __decode(bounds, number_of_bits, best)
print(f"Best solution: f({', '.join(str(x) for x in np.round(decoded, 3))}) = {np.round(score, 3)}")


Best solution: f(-2.0) = -2.667


**Define second minimizing function and run the algorithm**

In [9]:
def g(x: list) -> float:
    return 8 * (x[0] ** 3) - (24 * x[0] * x[1]) + x[1] ** 3

In [10]:
# define range for input
bounds = np.array([[-5.0, 5.0], [-5.0, 5.0]])

# define the total iterations
number_of_iterations = 500

# bits per variable
number_of_bits = 16

# define the population size
population_size = 500

# crossover rate
crossover_rate = 0.9

# mutation rate
mutation_rate = 1.0 / (float(number_of_bits) * len(bounds))

# perform the genetic algorithm search
best, score = genetic_algorithm(func=g, bounds=bounds, number_of_bits=number_of_bits, number_of_iterations=number_of_iterations, population_size=population_size,
                                crossover_rate=crossover_rate, mutation_rate=mutation_rate, verbose=False)

decoded = __decode(bounds, number_of_bits, best)
print(f"Best solution: g({', '.join(str(x) for x in np.round(decoded, 3))}) = {np.round(score, 3)}")


Best solution: g(-5.0, -5.0) = -1725.0
